In [1]:
import dimod


In [3]:
foods = {
    'rice' : {'Calories': 100, 'Protein': 3, 'Fat': 1, 'Carbs': 22, 'Fiber': 2, 
    'Taste': 7, 'Cost': 2.5, 'Units': 'continuous'},
    'tofu' : {'Calories': 140, 'Protein': 17, 'Fat': 9, 'Carbs': 3, 'Fiber': 2, 
    'Taste': 2, 'Cost': 4.0, 'Units': 'continuous'},
    'banana' : {'Calories': 90, 'Protein': 1, 'Fat': 0, 'Carbs': 23, 'Fiber': 3, 
    'Taste': 10, 'Cost': 1.0, 'Units': 'discrete'},
    'lentils' : {'Calories': 150, 'Protein': 9, 'Fat': 0, 'Carbs': 25, 'Fiber': 4, 
    'Taste': 3, 'Cost': 1.3, 'Units': 'continuous'},
    'bread' : {'Calories': 270, 'Protein': 9, 'Fat': 3, 'Carbs': 50, 'Fiber': 3, 
    'Taste': 5, 'Cost': 0.25, 'Units': 'continuous'},
    'avocado' : {'Calories': 300, 'Protein': 4, 'Fat': 30, 'Carbs': 20, 'Fiber': 14, 
    'Taste': 5, 'Cost': 2.0, 'Units': 'discrete'}
}

In [4]:
min_nutrients = {'Protein': 50, 'Fat': 30, 'Carbs': 130, 'Fiber': 30}
max_calories = 2000

## Variables
Instantiate some real and integer variables in a list, `quantities`, that in the solutions will be assigned values for the selected quantities of every available food.

In [5]:
quantities = [dimod.Real(f"{food}") if foods[food]["Units"] == 'continuous'
                                    else dimod.Integer(f"{food}")
                                    for food in foods.keys()]

Always keep in mind that such “variables” are actually `QuadraticModel` objects,

In [6]:
quantities[0]

QuadraticModel({'rice': 1.0}, {}, 0.0, {'rice': 'REAL'}, dtype='float64')

In [8]:
print(2*quantities[0])

QuadraticModel({'rice': 2.0}, {}, 0.0, {'rice': 'REAL'}, dtype='float64')


In [11]:
quantities[0] * quantities[1]

ValueError: REAL variables (e.g. 'rice') cannot have interactions

Bounds on the range of values for non-binary variables shrink the solution space the solver must search, so it is helpful to set such bounds; for many problems, you can find bounds from your knowledge of the problem. In this case, no food by itself should be assigned a quantity that exceeds `max_calories`.


In [12]:
for index, food in enumerate(foods.keys()):
    upper_bound = max_calories / foods[food]["Calories"]
    quantities[index].set_upper_bound(food, upper_bound)

The maximum quantity of rice, for example, which here has 100 calories per portion, is 20 portions because 20 * 100 = 2000

In [13]:
quantities[0].upper_bound("rice")

20.0

Lower bounds are actually required in this problem for correct formulation: a valid mathematical solution might be to offset the calories of gorging on large numbers of tasty bananas by eating a negative amount of high-in-calories bread, so the formulation must include the impossibility of consuming negative quantities of food. Because Ocean sets a default value of zero for ~dimod.quadratic.Real variables, no explicit configuration is needed.



You can now formulate an objective function and any constraints feasible solutions must meet, and set these in your CQM.

## Objective Function 
The objective function must maximize the taste of the diet plan while minimizing purchase cost. 
To maximise the taste and minimize the cost is to assign values to the variables that represtn quantities of each food, $q_{i}$, sucht that when multiplied by coefficients representing the cost, $c_{i}$, or taste $t_{i}$, of each food, form the liner terms of the following summations to be optimized: 

$\min \sum_{i} q_{i}c_{i}$ Minimize Cost <br>
$\max \sum_{i} q_{i}t_{i}$ Maximize Taste <br>


To optimize two differnt objectives, cost and taste, requires weighing on e against the other. A simpel way to do this, is to set priority weights. For example:

$ objective = \alpha(objective1) + \beta(objective2) $

By setting, for example $\alpha = 2$ and $beta = 1$, you double the priority of the first objective compared to the second. 

#### Instantiate the CQM

In [14]:
cqm = dimod.ConstrainedQuadraticModel()

A utility function `total_mix` can be defined to calculate the summations for any given category such as calories

In [15]:
def total_mix(quantity, category):
    return sum(q * c for q, c in zip(quantity, (foods[food][category] for food in foods.keys())))

## Set the Objective
Because Ocean solvers minimize objectives, to maximize taste, Taste is multiplied by -1 and minimized.



In [16]:
cqm.set_objective(-total_mix(quantities, "Taste") + 6*total_mix(quantities, "Cost"))

### Constraints:
1. Calories: no more than 2000
2. Protein: at least 50 
3. Fat: at least 30
4. Carbs: at least 130
5. Fiber: at least 30

Constrain the diet's calorie intake to the require daily maximum

In [17]:
cqm.add_constraint(total_mix(quantities, "Calories") <= max_calories, label = "Calories")

'Calories'

Require that the daily minimum of each nutrient is met or exceeded.


In [19]:
for nutrient, amount in min_nutrients.items():
    cqm.add_constraint(total_mix(quantities, nutrient) >= amount, label = nutrient)

In [20]:
list(cqm.constraints.keys())

['Calories', 'Protein', 'Fat', 'Carbs', 'Fiber']

In [21]:
print(cqm.constraints["Calories"].to_polystring())

100*rice + 140*tofu + 90*banana + 150*lentils + 270*bread + 300*avocado <= 2000.0


In [22]:
print(cqm.constraints["Protein"].to_polystring())

3*rice + 17*tofu + banana + 9*lentils + 9*bread + 4*avocado >= 50.0


### Solve the Problem by Sampling
Leap’s solvers can relieve you of the burden of any current and future development and optimization of hybrid algorithms that best solve your problem.

Ocean software’s dwave-system LeapHybridCQMSampler class enables you to easily incorporate Leap’s hybrid CQM solvers into your application:

In [23]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler()

In [26]:
sampleset = sampler.sample_cqm(cqm)
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
print("{} feasible solutions of {}.".format(len(feasible_sampleset), len(sampleset)))

69 feasible solutions of 113.


A utility function `print_diet` can be defined to display ther returned solutions in an intuitive way

In [34]:
def print_diet(sample):
    diet = {food: round(quantity, 1) for food, quantity in sample.items()}
    print(f"Diet: {diet}")
    taste_total = sum(foods[food]["Taste"] * amount for food, amount in sample.items())
    cost_total = sum(foods[food]["Cost"] * amount for food, amount in sample.items())
    print(f"Total Taste of {round(taste_total, 2)} at Cost {round(cost_total, 2)}")
    for constraint in cqm.iter_constraint_data(sample):
        print(f"{constraint.label} (nominal: {constraint.rhs_energy}): {round(constraint.lhs_energy)}")

In [35]:
best = feasible_sampleset.first.sample
print_diet(best)

Diet: {'avocado': 1.0, 'banana': 6.0, 'bread': 4.1, 'lentils': 0.3, 'rice': 0.0, 'tofu': 0.0}
Total Taste of 86.56 at Cost 9.46
Calories (nominal: 2000.0): 2000
Protein (nominal: 50.0): 50
Fat (nominal: 30.0): 42
Carbs (nominal: 130.0): 372
Fiber (nominal: 30.0): 46


## Tuning the Solution 
Consider sampling each part of the combined objective on its own (i.e. $\alpha = 1$ and $\beta = 0$ and $\alpha = 0$ and $\beta = 1$) and comparing the best solutions. Start with Taste:

In [36]:
cqm.set_objective(-total_mix(quantities, "Taste"))
sampleset_taste = sampler.sample_cqm(cqm)
feasible_sampleset_taste = sampleset_taste.filter(lambda row: row.is_feasible)
best_taste = feasible_sampleset_taste.first
print(round(best_taste.energy))

-177


In [37]:
print_diet(best_taste.sample)

Diet: {'avocado': 0.0, 'banana': 17.0, 'bread': 0.0, 'lentils': 0.0, 'rice': 0.0, 'tofu': 3.3}
Total Taste of 176.93 at Cost 30.41
Calories (nominal: 2000.0): 2000
Protein (nominal: 50.0): 74
Fat (nominal: 30.0): 30
Carbs (nominal: 130.0): 402
Fiber (nominal: 30.0): 58


Now for cost:

In [38]:
cqm.set_objective(total_mix(quantities, "Cost"))
sampleset_cost = sampler.sample_cqm(cqm)
feasible_sampleset_cost = sampleset_cost.filter(lambda row: row.is_feasible)
best_cost = feasible_sampleset_cost.first
print(round(best_cost.energy))

3


In [39]:
print_diet(best_cost.sample)

Diet: {'avocado': 1.0, 'banana': 0.0, 'bread': 5.3, 'lentils': 0.0, 'rice': 0.0, 'tofu': 0.0}
Total Taste of 31.67 at Cost 3.33
Calories (nominal: 2000.0): 1740
Protein (nominal: 50.0): 52
Fat (nominal: 30.0): 46
Carbs (nominal: 130.0): 287
Fiber (nominal: 30.0): 30


Because of the differences in energy scale between the two parts of the combined objective, , if you do not multiply the part representing cost by some positive factor, optimal solutions will maximize taste and neglect cost. That is, if in  you set set , solutions will likely be close or identical to those found when optimizing for taste alone.

In [40]:
cqm.set_objective(-total_mix(quantities, "Taste") + 1 * total_mix(quantities, "Cost"))
sampleset = sampler.sample_cqm(cqm)
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
best = feasible_sampleset.first
print(round(best.energy))

-147


In [43]:
print_diet(best.sample)

Diet: {'avocado': 0.0, 'banana': 17.0, 'bread': 0.0, 'lentils': 0.0, 'rice': 0.0, 'tofu': 3.3}
Total Taste of 176.93 at Cost 30.41
Calories (nominal: 2000.0): 2000
Protein (nominal: 50.0): 74
Fat (nominal: 30.0): 30
Carbs (nominal: 130.0): 402
Fiber (nominal: 30.0): 58


Compare the best solutions found when optimizing for taste and cost alone. Notice that to reduce 27 units of cost () in the latter solution, taste was decreased by 145 (), for a ratio of . To give each part of the combined objective a similar weighting, the Objective Function section above multiplied the part of the objective that minimizes cost by a factor of .

For low () ratios of 
 
 solutions are optimized for taste alone; for high ratios () solutions are optimized for cost. The relationship between this ratio and the weightings of the two parts of the combined optimization is non-linear, so while you can use such reasoning as was done above to find a starting point for “good” relative weightings, typically you need to experiment.

Notice that in all the previous solutions, the resulting diet relied on only two or three foods. If the dieter wants a more diverse diet, you can enforce that by setting appropriate bounds on the variables (or, equivalently, adding constraints on minimum/maximum quantities of each food).

In [46]:
cqm.set_objective(-total_mix(quantities, "Taste") + 6*total_mix(quantities, "Cost"))
for variable in cqm.variables:
    cqm.set_lower_bound(variable, 1)
sampleset_diverse = sampler.sample_cqm(cqm)
feasible_sampleset_diverse = sampleset_diverse.filter(lambda row: row.is_feasible)
best_diverse = feasible_sampleset_diverse.first.sample
print_diet(best_diverse)

ValueError: conflicting lower bounds: 'rice'